<a href="https://colab.research.google.com/github/hyeonwoongson/hackathon/blob/main/holt-winters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

훈련 데이터를 새로운 데이터 프레임으로 변환

In [ ]:
import pandas as pd
import re

from google.colab import drive
import pandas as pd

# 1. Google Drive 마운트
drive.mount('/content/drive')

# 2. 파일 경로 지정 및 데이터 불러오기
#    - 파일에 헤더가 있는 경우 names 파라미터는 사용하지 않습니다.
file_path = '/content/drive/MyDrive/hackathon_data/train.csv'
df = pd.read_csv(file_path)

# Inspect the data
print(df.head())
print(df.info())

# Ensure column names are correct
df.columns = ['영업일자', '영업장명_메뉴명', '매출수량']

# Convert '영업일자' to datetime objects
df['영업일자'] = pd.to_datetime(df['영업일자'])

# Extract 'year', 'month', 'day'
df['year'] = df['영업일자'].dt.year
df['month'] = df['영업일자'].dt.month
df['day'] = df['영업일자'].dt.day

# Create 'yoil' (day of week) column
day_of_week_map = {
    'Monday': '월요일',
    'Tuesday': '화요일',
    'Wednesday': '수요일',
    'Thursday': '목요일',
    'Friday': '금요일',
    'Saturday': '토요일',
    'Sunday': '일요일'
}
df['yoil'] = df['영업일자'].dt.day_name().map(day_of_week_map)

# Split '영업장명_메뉴명' into 'restaurant' and 'menu'
df[['restaurant', 'menu']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)

# Rename '매출수량' to 'sum'
df = df.rename(columns={'매출수량': 'sum'})

# Create the new DataFrame with the required columns
new_df = df[['year', 'month', 'day', 'yoil', 'restaurant', 'menu', 'sum']]

# Display the new DataFrame's head
print(new_df.head())

# Save the new DataFrame to a CSV file
new_df.to_csv('processed_data.csv', index=False, encoding='utf-8-sig')

print('New DataFrame saved to processed_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         영업일자            영업장명_메뉴명  매출수량
0  2023-01-01  느티나무 셀프BBQ_1인 수저세트     0
1  2023-01-02  느티나무 셀프BBQ_1인 수저세트     0
2  2023-01-03  느티나무 셀프BBQ_1인 수저세트     0
3  2023-01-04  느티나무 셀프BBQ_1인 수저세트     0
4  2023-01-05  느티나무 셀프BBQ_1인 수저세트     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   영업일자      102676 non-null  object
 1   영업장명_메뉴명  102676 non-null  object
 2   매출수량      102676 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.4+ MB
None
   year  month  day yoil  restaurant     menu  sum
0  2023      1    1  일요일  느티나무 셀프BBQ  1인 수저세트    0
1  2023      1    2  월요일  느티나무 셀프BBQ  1인 수저세트    0
2  2023      1    3  화요일  느티나무 셀프BBQ  1인 수저세트    0
3  2023      1    4  수요일  느티나무 셀프BBQ  1인 수저세트    0
4  2023      1  

테스트 데이터 하나의 파일로 모으기

In [ ]:
import pandas as pd
import glob
import os

# Get a list of all TEST_xx.csv files.
# 파일 경로가 올바른지 다시 한번 확인해주세요.
file_list = sorted(glob.glob('/content/drive/MyDrive/hackathon_data/TEST_*.csv'))

if not file_list:
    print("경로에 TEST_*.csv 파일이 없습니다. 경로를 다시 확인해주세요.")
else:
    # Create an empty list to store dataframes.
    df_list = []

    # Read each csv file and append to the list.
    for file in file_list:
        try:
            # 인코딩 오류 방지를 위해 utf-8로 지정합니다.
            df_list.append(pd.read_csv(file, encoding='utf-8'))
        except UnicodeDecodeError:
            print(f"파일 {file}을 utf-8로 디코딩할 수 없습니다. 다른 인코딩을 시도합니다.")
            df_list.append(pd.read_csv(file, encoding='cp949'))

    # Concatenate all dataframes into a single one.
    df_combined = pd.concat(df_list, ignore_index=True)

    # Define a dictionary to map English day names to Korean day names.
    yoil_map = {
        'Monday': '월요일',
        'Tuesday': '화요일',
        'Wednesday': '수요일',
        'Thursday': '목요일',
        'Friday': '금요일',
        'Saturday': '토요일',
        'Sunday': '일요일'
    }

    # Convert '영업일자' to datetime, 유효하지 않은 날짜는 NaT로 변환합니다.
    df_combined['영업일자'] = pd.to_datetime(df_combined['영업일자'], errors='coerce')

    # Extract 'year', 'month', 'day', 'yoil'.
    df_combined['year'] = df_combined['영업일자'].dt.year
    df_combined['month'] = df_combined['영업일자'].dt.month
    df_combined['day'] = df_combined['영업일자'].dt.day
    df_combined['yoil'] = df_combined['영업일자'].dt.day_name().map(yoil_map)

    # Split '영업장명_메뉴명' into 'restaurant' and 'menu'.
    df_combined[['restaurant', 'menu']] = df_combined['영업장명_메뉴명'].str.split('_', expand=True)

    # Rename '매출수량' to 'sum'.
    df_combined = df_combined.rename(columns={'매출수량': 'sum'})

    # Create a new DataFrame with the required columns.
    df_new = df_combined[['year', 'month', 'day', 'yoil', 'restaurant', 'menu', 'sum']]

    # df_new의 정보 확인 (진단용)
    print("--- df_new의 정보 확인 ---")
    df_new.info()

    # df_new의 처음 5개 행 미리보기 (진단용)
    print("\n--- df_new의 처음 5개 행 미리보기 ---")
    print(df_new.head().to_markdown(index=False, numalign="left", stralign="left"))

    # Save the new DataFrame to a CSV file.
    # 인코딩을 'utf-8-sig'로 지정하여 엑셀 호환성을 높입니다.
    df_new.to_csv('transformed_test_data.csv', index=False, encoding='utf-8-sig')
    print("\n'transformed_test_data.csv' 파일이 성공적으로 저장되었습니다.")

--- df_new의 정보 확인 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54040 entries, 0 to 54039
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        54040 non-null  int32 
 1   month       54040 non-null  int32 
 2   day         54040 non-null  int32 
 3   yoil        54040 non-null  object
 4   restaurant  54040 non-null  object
 5   menu        54040 non-null  object
 6   sum         54040 non-null  int64 
dtypes: int32(3), int64(1), object(3)
memory usage: 2.3+ MB

--- df_new의 처음 5개 행 미리보기 ---
| year   | month   | day   | yoil   | restaurant       | menu         | sum   |
|:-------|:--------|:------|:-------|:-----------------|:-------------|:------|
| 2024   | 6       | 16    | 일요일 | 느티나무 셀프BBQ | 1인 수저세트 | 2     |
| 2024   | 6       | 17    | 월요일 | 느티나무 셀프BBQ | 1인 수저세트 | 0     |
| 2024   | 6       | 18    | 화요일 | 느티나무 셀프BBQ | 1인 수저세트 | 0     |
| 2024   | 6       | 19    | 수요일 | 느티나무 셀프BBQ | 1인 수저세트 | 0   

훈련 데이터 경로 지정

In [ ]:
import os
import shutil
from google.colab import drive

# 1. 구글 드라이브 마운트
print("구글 드라이브를 마운트합니다...")
drive.mount('/content/drive')

# 2. 파일 경로 지정 (원본 파일명과 목표 파일명)
source_file = 'processed_data.csv'
destination_dir = '/content/drive/MyDrive/hackathon_data/'
destination_file = 'processed_data.csv'

# 3. 목표 폴더가 없으면 생성
if not os.path.exists(destination_dir):
    print(f"'{destination_dir}' 폴더를 생성합니다.")
    os.makedirs(destination_dir)

# 4. 파일 이동 및 이름 변경
source_path = os.path.join(os.getcwd(), source_file)
destination_path = os.path.join(destination_dir, destination_file)

try:
    shutil.move(source_path, destination_path)
    print(f"'{source_file}' 파일이 '{destination_path}' 로 성공적으로 이동되었습니다.")
except FileNotFoundError:
    print(f"오류: '{source_file}' 파일을 찾을 수 없습니다. 현재 폴더에 파일이 있는지 확인해 주세요.")

print("\n파일 이동 작업이 완료되었습니다.")

구글 드라이브를 마운트합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'processed_data.csv' 파일이 '/content/drive/MyDrive/hackathon_data/processed_data.csv' 로 성공적으로 이동되었습니다.

파일 이동 작업이 완료되었습니다.


훈련 데이터 영업장 기준으로 나누기

In [ ]:
import pandas as pd
import re

# 이전 단계에서 합쳐진 데이터프레임 다시 생성
df1 = pd.read_csv('/content/drive/MyDrive/hackathon_data/processed_data.csv')

# 'restaurant' 열의 고유한 값 확인
restaurants = df1['restaurant'].unique()

# 각 레스토랑별로 데이터프레임을 분리하고 파일로 저장
for restaurant in restaurants:
    # 파일 이름에 사용할 수 있도록 특수문자나 공백을 '_'로 대체
    # 파일명에 사용할 수 없는 문자 (\ / : * ? " < > |)를 제거합니다.
    safe_name = re.sub(r'[\\/:*?"<>|]', '', restaurant).replace(' ', '_')

    # 해당 레스토랑의 데이터만 필터링
    restaurant_df = df1[df1['restaurant'] == restaurant].copy()

    # 새로운 파일 이름 생성
    filename = f"{safe_name}_data.csv"

    # CSV 파일로 저장 (인덱스 제외)
    # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩을 사용합니다.
    restaurant_df.to_csv(filename, index=False, encoding='utf-8-sig')

    print(f"'{restaurant}' 데이터를 '{filename}' 파일로 저장했습니다.")

print("\n모든 레스토랑 데이터가 개별 파일로 성공적으로 분리되었습니다.")

'느티나무 셀프BBQ' 데이터를 '느티나무_셀프BBQ_data.csv' 파일로 저장했습니다.
'담하' 데이터를 '담하_data.csv' 파일로 저장했습니다.
'라그로타' 데이터를 '라그로타_data.csv' 파일로 저장했습니다.
'미라시아' 데이터를 '미라시아_data.csv' 파일로 저장했습니다.
'연회장' 데이터를 '연회장_data.csv' 파일로 저장했습니다.
'카페테리아' 데이터를 '카페테리아_data.csv' 파일로 저장했습니다.
'포레스트릿' 데이터를 '포레스트릿_data.csv' 파일로 저장했습니다.
'화담숲주막' 데이터를 '화담숲주막_data.csv' 파일로 저장했습니다.
'화담숲카페' 데이터를 '화담숲카페_data.csv' 파일로 저장했습니다.

모든 레스토랑 데이터가 개별 파일로 성공적으로 분리되었습니다.


In [ ]:
import os
import shutil
from google.colab import drive

# 1. 구글 드라이브 마운트
print("구글 드라이브를 마운트합니다...")
drive.mount('/content/drive')

# 2. 파일 경로 지정 (원본 파일명과 목표 파일명)
source_file = 'transformed_test_data.csv'
destination_dir = '/content/drive/MyDrive/hackathon_data/'
destination_file = 'transformed_test_data.csv'

# 3. 목표 폴더가 없으면 생성
if not os.path.exists(destination_dir):
    print(f"'{destination_dir}' 폴더를 생성합니다.")
    os.makedirs(destination_dir)

# 4. 파일 이동 및 이름 변경
source_path = os.path.join(os.getcwd(), source_file)
destination_path = os.path.join(destination_dir, destination_file)

try:
    shutil.move(source_path, destination_path)
    print(f"'{source_file}' 파일이 '{destination_path}' 로 성공적으로 이동되었습니다.")
except FileNotFoundError:
    print(f"오류: '{source_file}' 파일을 찾을 수 없습니다. 현재 폴더에 파일이 있는지 확인해 주세요.")

print("\n파일 이동 작업이 완료되었습니다.")

구글 드라이브를 마운트합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'transformed_test_data.csv' 파일이 '/content/drive/MyDrive/hackathon_data/transformed_test_data.csv' 로 성공적으로 이동되었습니다.

파일 이동 작업이 완료되었습니다.


테스트 데이터 경로 지정

In [ ]:
import os
import shutil
from google.colab import drive
import glob

# 1. 구글 드라이브 마운트
# 이 코드를 실행하면 인증 절차를 거쳐야 합니다.
print("구글 드라이브를 마운트합니다...")
drive.mount('/content/drive')

# 2. 파일들을 옮길 목표 경로 지정
# 드라이브 내의 'My Drive'는 '/content/drive/MyDrive' 경로에 연결됩니다.
destination_dir = '/content/drive/MyDrive/hackathon_data/train'

# 3. 목표 폴더가 없으면 생성
if not os.path.exists(destination_dir):
    print(f"'{destination_dir}' 폴더를 생성합니다.")
    os.makedirs(destination_dir)

# 4. 현재 폴더에 있는 '_data.csv'로 끝나는 모든 파일 찾기
source_files = glob.glob('*_data.csv')

# 이동할 파일 목록을 출력하여 확인
print(f"이동할 파일 목록: {source_files}")
print(f"총 {len(source_files)}개의 파일을 찾았습니다.")

# 5. 각 파일을 목표 폴더로 이동
for file_name in source_files:
    source_path = os.path.join(os.getcwd(), file_name)
    destination_path = os.path.join(destination_dir, file_name)

    # 파일 이동
    shutil.move(source_path, destination_path)
    print(f"'{file_name}' -> '{destination_dir}' 로 이동 완료.")

print("\n모든 파일 이동이 성공적으로 완료되었습니다.")

구글 드라이브를 마운트합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
이동할 파일 목록: ['라그로타_data.csv', '미라시아_data.csv', '연회장_data.csv', '카페테리아_data.csv', 'transformed_test_data.csv', 'processed_data.csv', '화담숲주막_data.csv', '담하_data.csv', '포레스트릿_data.csv', '화담숲카페_data.csv', '느티나무_셀프BBQ_data.csv']
총 11개의 파일을 찾았습니다.
'라그로타_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'미라시아_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'연회장_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'카페테리아_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'transformed_test_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'processed_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'화담숲주막_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'담하_data.csv' -> '/content/drive/MyDrive/hackathon_data/train' 로 이동 완료.
'포레

테스트 데이터 영업장 기준으로 나누기

In [ ]:
import pandas as pd
import re

try:
    # 1. 'transformed_test_data.csv' 파일 불러오기
    df = pd.read_csv('/content/drive/MyDrive/hackathon_data/transformed_test_data.csv')
    print("파일을 성공적으로 불러왔습니다.")

    # 2. 'restaurant' 열의 고유한 값 확인
    restaurants = df['restaurant'].unique()
    print(f"고유한 레스토랑의 수: {len(restaurants)}")
    print(f"레스토랑 목록: {list(restaurants)}")

    # 3. 각 레스토랑별로 데이터프레임을 분리하고 파일로 저장
    for restaurant in restaurants:
        # 파일 이름에 사용할 수 있도록 특수문자나 공백을 '_'로 대체
        safe_name = re.sub(r'[\\/:*?"<>|]', '', restaurant).replace(' ', '_')

        # 해당 레스토랑의 데이터만 필터링
        restaurant_df = df[df['restaurant'] == restaurant].copy()

        # 새로운 파일 이름 생성
        filename = f"{safe_name}_data.csv"

        # CSV 파일로 저장 (인덱스 제외)
        restaurant_df.to_csv(filename, index=False, encoding='utf-8-sig')

        print(f"'{restaurant}' 데이터를 '{filename}' 파일로 저장했습니다.")

    print("\n모든 레스토랑 데이터가 개별 파일로 성공적으로 분리되었습니다.")

except FileNotFoundError:
    print("오류: 'transformed_test_data.csv' 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")

파일을 성공적으로 불러왔습니다.
고유한 레스토랑의 수: 9
레스토랑 목록: ['느티나무 셀프BBQ', '담하', '라그로타', '미라시아', '연회장', '카페테리아', '포레스트릿', '화담숲주막', '화담숲카페']
'느티나무 셀프BBQ' 데이터를 '느티나무_셀프BBQ_data.csv' 파일로 저장했습니다.
'담하' 데이터를 '담하_data.csv' 파일로 저장했습니다.
'라그로타' 데이터를 '라그로타_data.csv' 파일로 저장했습니다.
'미라시아' 데이터를 '미라시아_data.csv' 파일로 저장했습니다.
'연회장' 데이터를 '연회장_data.csv' 파일로 저장했습니다.
'카페테리아' 데이터를 '카페테리아_data.csv' 파일로 저장했습니다.
'포레스트릿' 데이터를 '포레스트릿_data.csv' 파일로 저장했습니다.
'화담숲주막' 데이터를 '화담숲주막_data.csv' 파일로 저장했습니다.
'화담숲카페' 데이터를 '화담숲카페_data.csv' 파일로 저장했습니다.

모든 레스토랑 데이터가 개별 파일로 성공적으로 분리되었습니다.


테스트 데이터 경로 지정

In [ ]:
import os
import shutil
from google.colab import drive
import glob

# 1. 구글 드라이브 마운트
print("구글 드라이브를 마운트합니다...")
drive.mount('/content/drive')

# 2. 파일들을 옮길 목표 경로 지정
# 'train'이 아닌 'test' 폴더로 이동합니다.
destination_dir = '/content/drive/MyDrive/hackathon_data/test'

# 3. 목표 폴더가 없으면 생성
if not os.path.exists(destination_dir):
    print(f"'{destination_dir}' 폴더를 생성합니다.")
    os.makedirs(destination_dir)

# 4. 현재 폴더에 있는 '_data.csv'로 끝나는 모든 파일 찾기
source_files = glob.glob('*_data.csv')

# 이동할 파일 목록을 출력하여 확인
print(f"이동할 파일 목록: {source_files}")
print(f"총 {len(source_files)}개의 파일을 찾았습니다.")

# 5. 각 파일을 목표 폴더로 이동
for file_name in source_files:
    source_path = os.path.join(os.getcwd(), file_name)
    destination_path = os.path.join(destination_dir, file_name)

    # 파일 이동
    shutil.move(source_path, destination_path)
    print(f"'{file_name}' -> '{destination_dir}' 로 이동 완료.")

print("\n모든 파일 이동이 성공적으로 완료되었습니다.")

구글 드라이브를 마운트합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/MyDrive/hackathon_data/test' 폴더를 생성합니다.
이동할 파일 목록: ['라그로타_data.csv', '미라시아_data.csv', '연회장_data.csv', '카페테리아_data.csv', '화담숲주막_data.csv', '담하_data.csv', '포레스트릿_data.csv', '화담숲카페_data.csv', '느티나무_셀프BBQ_data.csv']
총 9개의 파일을 찾았습니다.
'라그로타_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'미라시아_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'연회장_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'카페테리아_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'화담숲주막_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'담하_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'포레스트릿_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'화담숲카페_data.csv' -> '/content/drive/MyDrive/hackathon_data/test' 로 이동 완료.
'느티나무_셀프BBQ_data.csv' 

holt-winters 모델(느티나무)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/느티나무_셀프BBQ_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/느티나무_셀프BBQ_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 23개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '1인 수저세트' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    5.090443
2024-06-17    1.892980
2024-06-18    1.959610
2024-06-19    2.090763
2024-06-20    3.473463
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    2.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    0.0
2024-06-20    4.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 31.63
  RMSE (Root Mean Squared Error): 5.62

--- 메뉴: '1인 수저세트' 예측 종료 ---


--- 메뉴: 'BBQ55(단체)' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16     35.605367
2024-06-17     58.193626
2024-06-18     54.802617
2024-06-19     67.540268
2024-06-20    106.345163
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16      0.0
2024-06-17      0.0
2024-06-18      0.0
2024-06-19     67.0
2024-06-20    124.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 5946.52
  RMSE (Root Mean Squared Error): 77.11

--- 메뉴: 'BBQ55(단체)' 예측 종료 ---


--- 메뉴: '대여료 30,000원' 예측 시작 ---
  예측값 (상위 5개): 
20

느티나무 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/느티나무_셀프BBQ_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/느티나무_셀프BBQ_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"느티나무_셀프BBQ_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델(담하)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/담하_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/담하_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 42개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '(단체) 공깃밥' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16   -0.056064
2024-06-17    0.822309
2024-06-18    0.505818
2024-06-19    0.915337
2024-06-20    1.048507
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    0.0
2024-06-17    0.0
2024-06-18    2.0
2024-06-19    3.0
2024-06-20    4.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 4.08
  RMSE (Root Mean Squared Error): 2.02

--- 메뉴: '(단체) 공깃밥' 예측 종료 ---


--- 메뉴: '(단체) 생목살 김치전골 2.0' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16     4.143922
2024-06-17    10.062939
2024-06-18    19.231363
2024-06-19    20.015807
2024-06-20    18.837762
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16      0.0
2024-06-17      0.0
2024-06-18      0.0
2024-06-19    129.0
2024-06-20     17.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 672.71
  RMSE (Root Mean Squared Error): 25.94

--- 메뉴: '(단체) 생목살 김치전골 2.0' 예측 종료 ---


--- 메뉴: '(단체) 은이버섯 갈비탕' 예측 시작 ---
  예측값

담하 영업장의 submission 파일 생성

> Add blockquote



In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/담하_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/담하_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"담하_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (라그로타)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/라그로타_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/라그로타_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 25개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: 'AUS (200g)' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    1.462362
2024-06-17   -0.022658
2024-06-18    1.791675
2024-06-19    1.819386
2024-06-20    1.768611
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    1.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    1.0
2024-06-20    0.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 5.05
  RMSE (Root Mean Squared Error): 2.25

--- 메뉴: 'AUS (200g)' 예측 종료 ---


--- 메뉴: 'G-Charge(3)' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    0.425447
2024-06-17    0.068879
2024-06-18    0.279551
2024-06-19    0.688187
2024-06-20    0.806526
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    0.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    0.0
2024-06-20    0.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 1.09
  RMSE (Root Mean Squared Error): 1.05

--- 메뉴: 'G-Charge(3)' 예측 종료 ---


--- 메뉴: 'Gls.Sileni' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    1.34

담하 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/라그로타_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/라그로타_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"라그로타_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (미라시아)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/미라시아_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/미라시아_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 31개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '(단체)브런치주중 36,000' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16     3.767847
2024-06-17    17.451224
2024-06-18    47.003344
2024-06-19    31.634874
2024-06-20    49.953013
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16      0.0
2024-06-17      0.0
2024-06-18     25.0
2024-06-19     32.0
2024-06-20    161.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 2278.13
  RMSE (Root Mean Squared Error): 47.73

--- 메뉴: '(단체)브런치주중 36,000' 예측 종료 ---


--- 메뉴: '(오븐) 하와이안 쉬림프 피자' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    0.674469
2024-06-17    0.543125
2024-06-18    0.556196
2024-06-19    0.358633
2024-06-20    0.595483
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    0.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    0.0
2024-06-20    0.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 1.94
  RMSE (Root Mean Squared Error): 1.39

--- 메뉴: '(오븐) 하와이안 쉬림프 피자' 예측 종료 ---


--- 메뉴: '(화덕) 불고기 페퍼로니 반

미라시아 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/미라시아_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/미라시아_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"미라시아_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (연회장)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/연회장_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/연회장_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 23개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: 'Cass Beer' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    -3.636010
2024-06-17     9.337765
2024-06-18    18.259328
2024-06-19    10.614613
2024-06-20    36.063030
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16      0.0
2024-06-17      0.0
2024-06-18      0.0
2024-06-19      0.0
2024-06-20    136.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 1545.95
  RMSE (Root Mean Squared Error): 39.32

--- 메뉴: 'Cass Beer' 예측 종료 ---


--- 메뉴: 'Conference L1' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16   -0.059432
2024-06-17    0.230248
2024-06-18    0.308824
2024-06-19    0.295780
2024-06-20    0.546229
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    0.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    1.0
2024-06-20    1.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 0.23
  RMSE (Root Mean Squared Error): 0.48

--- 메뉴: 'Conference L1' 예측 종료 ---


--- 메뉴: 'Conference L2' 예측 시작 ---
  예측값 (상위 

연회장 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/연회장_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/연회장_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"연회장_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (카페테리아)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/카페테리아_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/카페테리아_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 24개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '공깃밥(추가)' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    8.559828
2024-06-17    1.849007
2024-06-18   -2.322145
2024-06-19   -3.782312
2024-06-20   -2.769287
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16     7.0
2024-06-17     0.0
2024-06-18     3.0
2024-06-19     2.0
2024-06-20    17.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 2049.58
  RMSE (Root Mean Squared Error): 45.27

--- 메뉴: '공깃밥(추가)' 예측 종료 ---


--- 메뉴: '구슬아이스크림' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16   -0.371988
2024-06-17   -0.439488
2024-06-18   -0.520096
2024-06-19   -0.619189
2024-06-20   -0.760025
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    0.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    0.0
2024-06-20    0.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 1812.74
  RMSE (Root Mean Squared Error): 42.58

--- 메뉴: '구슬아이스크림' 예측 종료 ---


--- 메뉴: '단체식 13000(신)' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    30.

카페테리아 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/카페테리아_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/카페테리아_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"카페테리아_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (포레스트릿)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/포레스트릿_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/포레스트릿_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 12개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '꼬치어묵' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    4.561410
2024-06-17   -5.589221
2024-06-18   -6.357947
2024-06-19   -6.530669
2024-06-20   -5.491939
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    0.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    0.0
2024-06-20    0.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 99471.18
  RMSE (Root Mean Squared Error): 315.39

--- 메뉴: '꼬치어묵' 예측 종료 ---


--- 메뉴: '떡볶이' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    8.853808
2024-06-17   -3.576116
2024-06-18   -3.857501
2024-06-19   -3.750216
2024-06-20    0.926445
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    5.0
2024-06-17    0.0
2024-06-18    0.0
2024-06-19    0.0
2024-06-20    0.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 32069.50
  RMSE (Root Mean Squared Error): 179.08

--- 메뉴: '떡볶이' 예측 종료 ---


--- 메뉴: '복숭아 아이스티' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16     3.135355
2024-06-17  

포레스트릿 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/포레스트릿_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/포레스트릿_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"포레스트릿_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (화담숲주막)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/화담숲주막_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/화담숲주막_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 8개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '느린마을 막걸리' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    46.438598
2024-06-17   -16.178481
2024-06-18     7.850430
2024-06-19    11.373002
2024-06-20    21.768943
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    33.0
2024-06-17     0.0
2024-06-18     9.0
2024-06-19    10.0
2024-06-20    19.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 2441.34
  RMSE (Root Mean Squared Error): 49.41

--- 메뉴: '느린마을 막걸리' 예측 종료 ---


--- 메뉴: '단호박 식혜 ' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    25.887446
2024-06-17     5.398490
2024-06-18    14.082782
2024-06-19    10.920963
2024-06-20    12.957177
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    33.0
2024-06-17     0.0
2024-06-18     8.0
2024-06-19     9.0
2024-06-20     5.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 739.03
  RMSE (Root Mean Squared Error): 27.19

--- 메뉴: '단호박 식혜 ' 예측 종료 ---


--- 메뉴: '병천순대' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16

화담숲주막 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/화담숲주막_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/화담숲주막_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"화담숲주막_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

holt-winters 모델 (화담숲카페)

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. 데이터 불러오기
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/화담숲카페_data.csv')
    test_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/화담숲카페_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 고유한 메뉴 목록 추출
unique_menus = test_df['menu'].unique()
print(f"\n총 {len(unique_menus)}개의 메뉴에 대해 예측을 시작합니다.")

# 3. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 시작 ---")

    # 해당 메뉴에 대한 훈련/테스트 데이터 필터링
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    test_menu_df = test_df[test_df['menu'] == menu_item].copy()

    # 결측값 포함 행 제거 (날짜 생성 전)
    train_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)
    test_menu_df.dropna(subset=['year', 'month', 'day', 'sum'], inplace=True)

    # 날짜별 'sum' 값 합산 (중복 날짜 제거)
    train_agg = train_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()
    test_agg = test_menu_df.groupby(['year', 'month', 'day'])['sum'].sum().reset_index()

    if train_agg.empty or test_agg.empty:
        print(f"'{menu_item}' 메뉴의 데이터가 부족하여 예측을 건너뜁니다.")
        continue

    # 날짜 인덱스 생성
    train_agg['date'] = pd.to_datetime(train_agg[['year', 'month', 'day']])
    train_agg.set_index('date', inplace=True)

    test_agg['date'] = pd.to_datetime(test_agg[['year', 'month', 'day']])
    test_agg.set_index('date', inplace=True)

    # 시계열 데이터 선택
    train_series = train_agg['sum']
    test_series = test_agg['sum']

    # 날짜가 비어있는 경우 일별 빈도(D)로 채우기
    train_series = train_series.asfreq('D')
    test_series = test_series.asfreq('D')

    # Holt-Winters 모델 구현 및 훈련
    model = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_model = model.fit()

    # 예측
    forecast_steps = len(test_series)
    predictions = fitted_model.forecast(steps=forecast_steps)

    # 평가
    results_df = pd.DataFrame({
        'actual': test_series,
        'predicted': predictions
    }).dropna()

    if not results_df.empty:
        actual_values = results_df['actual']
        predictions_filtered = results_df['predicted']

        mse = mean_squared_error(actual_values, predictions_filtered)
        rmse = np.sqrt(mse)

        print(f"  예측값 (상위 5개): \n{predictions_filtered.head()}")
        print(f"\n  실제 테스트 데이터 (상위 5개): \n{actual_values.head()}")
        print(f"\n  MSE (Mean Squared Error): {mse:.2f}")
        print(f"  RMSE (Root Mean Squared Error): {rmse:.2f}")
    else:
        print("  예측과 실제값을 비교할 데이터가 부족하여 평가를 건너뜁니다.")

    print(f"\n--- 메뉴: '{menu_item}' 예측 종료 ---\n")

파일을 성공적으로 불러왔습니다.

총 5개의 메뉴에 대해 예측을 시작합니다.

--- 메뉴: '메밀미숫가루' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    64.941606
2024-06-17     5.277200
2024-06-18    40.842841
2024-06-19    38.373820
2024-06-20    54.655637
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    92.0
2024-06-17     0.0
2024-06-18    40.0
2024-06-19    41.0
2024-06-20    40.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 3035.48
  RMSE (Root Mean Squared Error): 55.10

--- 메뉴: '메밀미숫가루' 예측 종료 ---


--- 메뉴: '아메리카노 HOT' 예측 시작 ---
  예측값 (상위 5개): 
2024-06-16    11.016414
2024-06-17    -0.905652
2024-06-18    12.057338
2024-06-19    10.923838
2024-06-20    10.579959
Name: predicted, dtype: float64

  실제 테스트 데이터 (상위 5개): 
2024-06-16    12.0
2024-06-17     0.0
2024-06-18     6.0
2024-06-19     4.0
2024-06-20     5.0
Name: actual, dtype: float64

  MSE (Mean Squared Error): 2167.42
  RMSE (Root Mean Squared Error): 46.56

--- 메뉴: '아메리카노 HOT' 예측 종료 ---


--- 메뉴: '아메리카노 ICE' 예측 시작 ---
  예측값 (상위 5개): 
2024

화담숲카페 영업장의 submission 파일 생성

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings
from datetime import timedelta

# statsmodels 경고 무시
warnings.filterwarnings("ignore")

# 1. 데이터 불러오기 및 전처리
try:
    train_df = pd.read_csv('/content/drive/MyDrive/hackathon_data/train/화담숲카페_data.csv')
    test_df_original = pd.read_csv('/content/drive/MyDrive/hackathon_data/test/화담숲카페_data.csv')
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 날짜/시간 인덱스 생성
train_df['date'] = pd.to_datetime(train_df[['year', 'month', 'day']])
test_df_original['date'] = pd.to_datetime(test_df_original[['year', 'month', 'day']])

# 결측값 포함 행 제거
train_df.dropna(subset=['date', 'sum'], inplace=True)
test_df_original.dropna(subset=['date', 'sum'], inplace=True)

# 2. 예측 대상 날짜들 찾기 (모든 메뉴에 대해 공통)
# 테스트 데이터의 전체 날짜 범위
full_test_date_range = pd.date_range(
    start=test_df_original['date'].min(),
    end=test_df_original['date'].max(),
    freq='D'
)

# 테스트 데이터에 실제 값이 존재하는 날짜
dates_with_data = pd.to_datetime(test_df_original['date'].unique())

# 전체 날짜 범위에서 실제 데이터가 있는 날짜를 제외하여 빈 기간 식별
missing_dates = full_test_date_range.difference(dates_with_data)

# 마지막 7일 기간 추가 (테스트 데이터 마지막 날 다음 날부터)
last_test_date = test_df_original['date'].max()
final_week_dates = pd.date_range(
    start=last_test_date + timedelta(days=1),
    periods=7,
    freq='D'
)

# 예측할 날짜 전체 리스트 생성
forecast_dates = missing_dates.append(final_week_dates).sort_values()

# 예측할 날짜 목록 출력
print(f"\n총 {len(forecast_dates)}개의 날짜에 대해 예측을 수행합니다.")
print("예측 날짜 목록:")
print(forecast_dates)

# 3. 고유한 메뉴 목록 추출 및 예측
unique_menus = test_df_original['menu'].unique()
predictions_dict = {}

# 4. 메뉴별로 반복하여 예측 수행
for menu_item in unique_menus:
    print(f"\n--- 메뉴: '{menu_item}' 예측 진행 중... ---")

    # 해당 메뉴에 대한 훈련 데이터 필터링 및 합산
    train_menu_df = train_df[train_df['menu'] == menu_item].copy()
    train_series = train_menu_df.groupby('date')['sum'].sum().asfreq('D')

    if train_series.empty:
        predictions_dict[menu_item] = np.zeros(len(forecast_dates))
        print(f"  데이터 부족으로 '{menu_item}' 예측값을 0으로 처리합니다.")
        continue

    # Holt-Winters 모델 훈련
    model_hw = ExponentialSmoothing(
        train_series,
        trend='add',
        seasonal='add',
        seasonal_periods=7
    )
    fitted_hw = model_hw.fit()

    # 예측 수행
    menu_predictions_series = fitted_hw.predict(
        start=forecast_dates.min(),
        end=forecast_dates.max()
    )

    # 예측값을 빈 날짜에 맞게 필터링
    final_predictions_for_menu = menu_predictions_series.reindex(forecast_dates)

    # 예측값 저장
    predictions_dict[menu_item] = final_predictions_for_menu.values

# 5. 예측값으로 최종 데이터프레임 생성
final_predictions_df = pd.DataFrame(predictions_dict, index=forecast_dates)

# 열 라벨(column labels) 생성
column_labels = [f"화담숲카페_{menu}" for menu in unique_menus]
final_predictions_df.columns = column_labels

# 6. 예측 파일로 저장
file_name = 'test_predictions_by_date.csv'
final_predictions_df.to_csv(file_name, encoding='utf-8-sig', index=True)

print(f"\n성공적으로 예측이 완료되었습니다. 결과는 '{file_name}' 파일에 저장되었습니다.")

파일을 성공적으로 불러왔습니다.

총 70개의 날짜에 대해 예측을 수행합니다.
예측 날짜 목록:
DatetimeIndex(['2024-07-14', '2024-07-15', '2024-07-16', '2024-07-17',
               '2024-07-18', '2024-07-19', '2024-07-20', '2024-08-18',
               '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22',
               '2024-08-23', '2024-08-24', '2024-09-22', '2024-09-23',
               '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27',
               '2024-09-28', '2024-10-27', '2024-10-28', '2024-10-29',
               '2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2025-01-05',
               '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09',
               '2025-01-10', '2025-01-11', '2025-02-09', '2025-02-10',
               '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14',
               '2025-02-15', '2025-03-16', '2025-03-17', '2025-03-18',
               '2025-03

submission 파일들 경로 지정

In [ ]:
import os
import shutil
from google.colab import drive

# 1. 구글 드라이브 마운트
print("구글 드라이브를 마운트합니다...")
drive.mount('/content/drive')

# 2. 이동할 파일 목록 지정
# 옮기고자 하는 파일 이름을 아래 리스트에 추가해주세요.
files_to_move = [
    'test_predictions_느티나무.csv',
    'test_predictions_담하.csv',
    'test_predictions_라그로타.csv',
    'test_predictions_미라시아.csv',
    'test_predictions_연회장.csv',
    'test_predictions_카페테리아.csv',
    'test_predictions_포레스트릿.csv',
    'test_predictions_화담숲주막.csv',
    'test_predictions_화담숲카페.csv'
]

# 3. 파일들을 옮길 목표 경로 지정
destination_dir = '/content/drive/MyDrive/hackathon_data/submission'

# 4. 목표 폴더가 없으면 생성
if not os.path.exists(destination_dir):
    print(f"'{destination_dir}' 폴더를 생성합니다.")
    os.makedirs(destination_dir)

# 5. 각 파일을 목표 폴더로 이동
for file_name in files_to_move:
    source_path = os.path.join(os.getcwd(), file_name)
    destination_path = os.path.join(destination_dir, file_name)

    try:
        shutil.move(source_path, destination_path)
        print(f"'{file_name}' -> '{destination_dir}' 로 이동 완료.")
    except FileNotFoundError:
        print(f"오류: '{file_name}' 파일을 찾을 수 없습니다. 현재 폴더에 파일이 있는지 확인해 주세요.")

print("\n모든 파일 이동 작업이 완료되었습니다.")

구글 드라이브를 마운트합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/MyDrive/hackathon_data/submission' 폴더를 생성합니다.
'test_predictions_느티나무.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_담하.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_라그로타.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_미라시아.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_연회장.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_카페테리아.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_포레스트릿.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_화담숲주막.csv' -> '/content/drive/MyDrive/hackathon_data/submission' 로 이동 완료.
'test_predictions_화담숲카페.csv' -> '/content/drive/MyDrive/hackat

In [ ]:
import pandas as pd
import os

# 합칠 파일 목록 (요청하신 순서대로)
file_list = [
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_느티나무.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_담하.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_라그로타.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_미라시아.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_연회장.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_카페테리아.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_포레스트릿.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲주막.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲카페.csv'
]

# 합쳐진 결과를 저장할 빈 데이터프레임 초기화
combined_df = None

# 파일들을 하나씩 불러와서 합치기
for file in file_list:
    try:
        # 첫 번째 열을 인덱스로 지정하여 불러옴
        df = pd.read_csv(file, index_col=0)

        if combined_df is None:
            # 첫 번째 파일로 데이터프레임 초기화
            combined_df = df
        else:
            # 다음 파일들을 기존 데이터프레임에 열 단위로 합치기
            combined_df = pd.concat([combined_df, df], axis=1)

        print(f"'{file}' 파일을 성공적으로 합쳤습니다.")

    except FileNotFoundError:
        print(f"오류: '{file}' 파일을 찾을 수 없습니다. 건너뜁니다.")
    except Exception as e:
        print(f"'{file}' 파일 처리 중 오류 발생: {e}")

if combined_df is not None:
    # 최종 결과 확인
    print("\n--- 합쳐진 최종 데이터프레임 정보 ---")
    combined_df.info()
    print("\n--- 합쳐진 최종 데이터프레임 상위 5개 행 ---")
    print(combined_df.head())

    # 파일을 CSV로 저장
    combined_df.to_csv('ordered_combined_predictions.csv', encoding='utf-8-sig', index=True)
    print("\n'ordered_combined_predictions.csv' 파일이 성공적으로 생성되었습니다.")
else:
    print("\n합칠 수 있는 파일이 없어 최종 데이터프레임이 생성되지 않았습니다.")

'/content/drive/MyDrive/hackathon_data/submission/test_predictions_느티나무.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_담하.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_라그로타.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_미라시아.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_연회장.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_카페테리아.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_포레스트릿.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲주막.csv' 파일을 성공적으로 합쳤습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲카페.csv' 파일을 성공적으로 합쳤습니다.

--- 합쳐진 최종 데이터프레임 정보 ---
<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 2024-07-14 to 2025-05-31
Columns: 193 entries, 느티나무_셀프BBQ_1인 수저세트 to 화

In [ ]:
import pandas as pd
import os

# 합칠 파일 목록을 요청하신 순서대로 나열합니다.
file_order = [
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_느티나무.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_담하.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_라그로타.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_미라시아.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_연회장.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_카페테리아.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_포레스트릿.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲주막.csv',
    '/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲카페.csv'
]

# 합칠 데이터프레임을 담을 리스트
dfs_to_combine = []

# 파일들을 순서대로 불러와서 리스트에 추가합니다.
for file in file_order:
    try:
        df = pd.read_csv(file, index_col=0)
        dfs_to_combine.append(df)
        print(f"'{file}' 파일을 성공적으로 불러왔습니다.")
    except FileNotFoundError:
        print(f"오류: '{file}' 파일을 찾을 수 없습니다. 건너뜁니다.")
    except Exception as e:
        print(f"'{file}' 파일 처리 중 오류 발생: {e}")

if not dfs_to_combine:
    print("\n합칠 수 있는 파일이 없어 최종 데이터프레임이 생성되지 않았습니다.")
else:
    # 리스트에 담긴 데이터프레임들을 열(columns) 방향으로 합칩니다.
    # 순서가 보장됩니다.
    combined_df = pd.concat(dfs_to_combine, axis=1)

    # 최종 결과 확인
    print("\n--- 합쳐진 최종 데이터프레임 정보 ---")
    combined_df.info()
    print("\n--- 합쳐진 최종 데이터프레임 상위 5개 행 ---")
    print(combined_df.head())

    # 합쳐진 파일을 CSV로 저장
    output_filename = 'ordered_combined_predictions.csv'
    combined_df.to_csv(output_filename, encoding='utf-8-sig', index=True)
    print(f"\n'{output_filename}' 파일이 성공적으로 생성되었습니다.")

'/content/drive/MyDrive/hackathon_data/submission/test_predictions_느티나무.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_담하.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_라그로타.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_미라시아.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_연회장.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_카페테리아.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_포레스트릿.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲주막.csv' 파일을 성공적으로 불러왔습니다.
'/content/drive/MyDrive/hackathon_data/submission/test_predictions_화담숲카페.csv' 파일을 성공적으로 불러왔습니다.

--- 합쳐진 최종 데이터프레임 정보 ---
<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 2024-07-14 to 2025-05-31
Columns: 193 entries, 느티나무_셀프BBQ_1인 

In [ ]:
import pandas as pd

# 1. 파일 불러오기
try:
    df = pd.read_csv('ordered_combined_predictions.csv', index_col=0)
    print("파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 'ordered_combined_predictions.csv' 파일을 찾을 수 없습니다. 파일 이름을 확인해 주세요.")
    exit()

# 2. 새로운 행 라벨 생성
# TEST_00+1일 부터 TEST_09+7일 까지 총 70개의 라벨 생성
row_labels = []
for i in range(10):
    for j in range(1, 8):
        row_labels.append(f"TEST_{i:02d}+{j}일")

# 기존 데이터프레임의 행 수와 생성된 라벨의 수가 같은지 확인
if len(df) != len(row_labels):
    print(f"오류: 데이터프레임의 행 수({len(df)})와 생성된 라벨 수({len(row_labels)})가 일치하지 않습니다.")
    exit()

# 3. 데이터프레임의 인덱스를 새로운 라벨로 변경하고 이름 지정
df.index = row_labels
df.index.name = '영업일자'

# 4. 최종 결과 확인
print("\n--- 최종 데이터프레임 정보 ---")
df.info()
print("\n--- 최종 데이터프레임 상위 5개 행 ---")
print(df.head())

# 5. 파일을 CSV로 저장
output_filename = 'final_submission.csv'
df.to_csv(output_filename, encoding='utf-8-sig', index=True)
print(f"\n'{output_filename}' 파일이 성공적으로 생성되었습니다.")

파일을 성공적으로 불러왔습니다.

--- 최종 데이터프레임 정보 ---
<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, TEST_00+1일 to TEST_09+7일
Columns: 193 entries, 느티나무_셀프BBQ_1인 수저세트 to 화담숲카페_현미뻥스크림
dtypes: float64(193)
memory usage: 106.1+ KB

--- 최종 데이터프레임 상위 5개 행 ---
            느티나무_셀프BBQ_1인 수저세트  느티나무_셀프BBQ_BBQ55(단체)  느티나무_셀프BBQ_대여료 30,000원  \
영업일자                                                                           
TEST_00+1일            5.258979             38.640063                3.532704   
TEST_00+2일            2.061516             61.228322               -1.129349   
TEST_00+3일            2.128147             57.837314                0.376052   
TEST_00+4일            2.259300             70.574964                0.609066   
TEST_00+5일            3.642000            109.379859                3.840234   

            느티나무_셀프BBQ_대여료 60,000원  느티나무_셀프BBQ_대여료 90,000원  \
영업일자                                                         
TEST_00+1일                1.015318                0.444959   
TE